In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import IPython
from utils import myfun

ModuleNotFoundError: No module named 'utils'

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")


def triplets(sent,maxlen,maxl=256,length_penalty = 0):
    gen_kwargs = {
        "max_length": maxl,
        "length_penalty": length_penalty,
        "num_beams": 3,
        "num_return_sequences": 3,
    }
    model_inputs = tokenizer(sent, max_length=maxlen, padding=True, truncation=True, return_tensors = 'pt')
    
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
        **gen_kwargs,
    )
    
    # Extract text
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    
    # Extract triplets
    triplets = []
    for idx, sentence in enumerate(decoded_preds):
        triplets.extend(extract_triplets(sentence))  

    unique_data = []
    for d in triplets:
        # Check if the current dictionary is not already in the list of unique dictionaries
        if d not in unique_data:
            # If not, add it to the list of unique dictionaries
            unique_data.append(d)
    return unique_data

In [ ]:
model

In [10]:
sent = "Droṇa, the preceptor of the Dhārtarāṣṭras and the Pāṇḍavas, Dhṛṣṭadyumna, etc., son of Bharadvāja, husband of Kṛpī, and father of Aśvatthāman. (Anukram.): I (Ācāryaṃ).—(Parvasaṅgr.): I .—(do.): I .—(Ādivaṃśāvatāraṇa): I (father of Aśvatthāman), (Dhṛṣṭadyumna was born for the destruction of D.).—(Aṃśāvat.): I (Bhāradvājaṃ, born from a portion of Bṛhaspati), (Dhanurvede Vede ca yaṃ taṃ Vedavido viduḥ variṣṭhaṃ).—(Sambhavap.): Bhīṣma prevailed upon D. to accept the Pāṇḍavas and the Kauravas as his pupils in the Dhanurveda: I (Bhāradvājāya).—(do.): Janamejaya said: “Tell me about the birth of Droṇa and of his son Aśvatthāman.” Vaiśampāyana said: The ṛṣi Bharadvāja who dwelt at Gaṅgādvāra one day saw the Apsaras Ghṛtācī coming from her bath; then his seed came out, but was preserved by him in a vessel (droṇa); from that seed sprang D., who studied the Vedas and the Vedāṅgas. Bharadvāja had taught Agniveśa the use of the Āgneya weapon, and the muni who had sprung from Fire (i.e. Agniveśa) now taught it to D.: I .—(do.): D. was a friend of Drupada (q.v.). Droṇa remained in the hermitage of his father and practised austerities. With Kṛpī Śāradvatī he had the son Aśvatthāman (b). D. studied the Dhanurveda. About this time Rāma Jāmadagnya desired to distribute his wealth among the brahmans, and Droṇa (of the race of Aṅgiras, v. ) with his disciples set out for the mountain Mahendra to have his share in the wealth and learn from Rāma the Dhanurveda, the use of celestial weapons, and the nītiśāstra. When he arrived, Rāma had already given his wealth to the brahmans and the earth to Kaśyapa; he then asked D. to choose either his body or his weapons. Droṇa accepted the weapons and the Dhanurveda, and then repaired to Drupada: I .—(do.): As Drupada rebuked him for calling him his friend, D., filled with wrath, went to Hāstinapura, where he for some time lived privately in the house of Kṛpa (his brother-in-law), while Aśvatthāman, being unknown, taught the Pārthas (i.e. the Pāṇḍavas) the use of arms. One day, when D. had by means of blades of grass, which were consecrated by mantras, recovered a ball which the princes had lost in a well, and, by means of an arrow, his ring which he had thrown into the well, he was called to Bhīṣma, whom he told that he had applied to Drupada in order to obtain milk for Aśvatthāman (b), but had been rebuked by him. Bhīṣma asked D. to make the princes accomplished in arms: I .—(do.): Bhīṣma presented D. with a pretty house, etc., and gave the Dhārtarāṣṭras and the Pāṇḍavas to him as his pupils. Arjuna (b) became the foremost of D.'s pupils, even superior to Aśvatthāman (c). Also the Vṛṣṇis and the Andhakas and princes from various countries became his pupils. Even Karṇa came to learn the Dhanurveda; he supported by Duryodhana, frequently defied Arjuna: I .—(Ekalavya): I (D. asked the thumb of the right hand of Ekalavya as his tutorial fee, in order that Ekalavya might not surpass the Pāṇḍavas).—(Sambhavap.): The foremost pupils of Droṇa were Duryodhana, Bhīmasena, Aśvatthāman (b), Nakula (c) and Sahadeva (c), Yudhiṣṭhira (d), and Arjuna (e). The Dhārtarāṣṭras became very jealous of Bhīma and Arjuna: I ."
triplets(sent,256)

[{'head': 'Droṇa', 'type': 'father', 'tail': 'Bharadvāja'},
 {'head': 'Droṇa', 'type': 'spouse', 'tail': 'Kṛpī'},
 {'head': 'Droṇa', 'type': 'child', 'tail': 'Aśvatthāman'},
 {'head': 'Bharadvāja', 'type': 'child', 'tail': 'Droṇa'},
 {'head': 'Kṛpī', 'type': 'spouse', 'tail': 'Droṇa'},
 {'head': 'Aśvatthāman', 'type': 'father', 'tail': 'Droṇa'},
 {'head': 'Kṛpī', 'type': 'child', 'tail': 'Aśvatthāman'},
 {'head': 'Aśvatthāman', 'type': 'mother', 'tail': 'Kṛpī'}]

# Importing the cleaned text file and extracting triplets

In [2]:
texts = []

with open('cleanedTextFileFinalDONE.txt', 'r',encoding='utf-8') as file:
    for line in file:
        texts.append(line[:-1])
        


In [3]:
texts[5:10]

['Ābhāsurāḥ, a class of gods .',
 'Abhaya .',
 'Abhibhū, king of Kāśi Vibhu. = Kṛṣṇa, .',
 'Abhigamya = Śiva .',
 'Abhijit, the nd of the nakṣatras, when beginning with Aśvinī .']

In [36]:
relations = []
for text in texts[:20]:
    relations.extend(triplets(text,256))
    

In [37]:
relations

[{'head': 'Abala Āṅgirasa III',
  'type': 'instance of',
  'tail': 'yajñamuṣo devāḥ'},
 {'head': 'Abala Āṅgirasa III', 'type': 'creator', 'tail': 'Agni Pāñcajanya'},
 {'head': 'Rājadh XII', 'type': 'occupation', 'tail': 'ascetics'},
 {'head': 'Abbhakṣāḥ masc pl', 'type': 'occupation', 'tail': 'ascetics'},
 {'head': 'Rājadh XII', 'type': 'occupation', 'tail': 'ascetic'},
 {'head': 'Nārāyaṇīya XII', 'type': 'instance of', 'tail': 'Mahāpuruṣastava'},
 {'head': 'Nārāyaṇīya XII', 'type': 'part of', 'tail': 'Mahāpuruṣastava'},
 {'head': 'Mahāpuruṣastava', 'type': 'has part', 'tail': 'Nārāyaṇīya XII'},
 {'head': 'Abhagnaparisaṅkhyāna',
  'type': 'instance of',
  'tail': 'Mahāpuruṣastava'},
 {'head': 'inter rupted', 'type': 'facet of', 'tail': 'Abhagnayoga'},
 {'head': 'Abhagnayoga', 'type': 'subclass of', 'tail': 'contemplation'},
 {'head': 'inter rupted', 'type': 'part of', 'tail': 'Abhagnayoga'},
 {'head': 'Abhāsura No. shining', 'type': 'part of', 'tail': 'BR'},
 {'head': 'BR', 'type': 'ha

In [39]:
relations = []
for text in texts[:20]:
    relations.extend(triplets(text,512))
    

In [40]:
relations

[{'head': 'Abala Āṅgirasa III',
  'type': 'instance of',
  'tail': 'yajñamuṣo devāḥ'},
 {'head': 'Abala Āṅgirasa III', 'type': 'creator', 'tail': 'Agni Pāñcajanya'},
 {'head': 'Rājadh XII', 'type': 'occupation', 'tail': 'ascetics'},
 {'head': 'Abbhakṣāḥ masc pl', 'type': 'occupation', 'tail': 'ascetics'},
 {'head': 'Rājadh XII', 'type': 'occupation', 'tail': 'ascetic'},
 {'head': 'Nārāyaṇīya XII', 'type': 'instance of', 'tail': 'Mahāpuruṣastava'},
 {'head': 'Nārāyaṇīya XII', 'type': 'part of', 'tail': 'Mahāpuruṣastava'},
 {'head': 'Mahāpuruṣastava', 'type': 'has part', 'tail': 'Nārāyaṇīya XII'},
 {'head': 'Abhagnaparisaṅkhyāna',
  'type': 'instance of',
  'tail': 'Mahāpuruṣastava'},
 {'head': 'inter rupted', 'type': 'facet of', 'tail': 'Abhagnayoga'},
 {'head': 'Abhagnayoga', 'type': 'subclass of', 'tail': 'contemplation'},
 {'head': 'inter rupted', 'type': 'part of', 'tail': 'Abhagnayoga'},
 {'head': 'Abhāsura No. shining', 'type': 'part of', 'tail': 'BR'},
 {'head': 'BR', 'type': 'ha

In [38]:
# Code to write all the relations into a CSV file

import csv
# Specify the field names for the CSV file
fieldnames = ['head', 'type', 'tail']

# Write the data to a CSV file
with open('triplets.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header
    writer.writeheader()
    
    # Write each dictionary as a row in the CSV file
    for row in relations:
        writer.writerow(row)

In [9]:
tmp = "Aśvatthāman2, an elephant belonging to the Mālava king Indravarman and killed by Bhīma to delude Droṇa and make him believe that his son had been killed. § 602 (Droṇavadhap.): VII, 190, 8708—10, 8744.—§ 603 (Nārā- yaṇāstramokṣap.): 193, 8946, 8948.—§ 630 (Rājadh.): XII, 27, 816."
triplets(tmp,256)

[{'head': 'Indravarman', 'type': 'child', 'tail': 'Droṇa'},
 {'head': 'Aśvatthāman2', 'type': 'instance of', 'tail': 'elephant'},
 {'head': 'Droṇa', 'type': 'father', 'tail': 'Indravarman'}]

In [10]:
tmp = "Ādideva1 the first god = the Sun, the th name of the Sun in the enumeration of Dhaumya"
triplets(tmp,256)


[{'head': 'Sun', 'type': 'said to be the same as', 'tail': 'Dhaumya'},
 {'head': 'Dhaumya', 'type': 'said to be the same as', 'tail': 'Sun'},
 {'head': 'Sun', 'type': 'part of', 'tail': 'Dhaumya'},
 {'head': 'Dhaumya', 'type': 'has part', 'tail': 'Sun'},
 {'head': 'Ādideva', 'type': 'said to be the same as', 'tail': 'Sun'},
 {'head': 'Sun', 'type': 'said to be the same as', 'tail': 'Ādideva'}]

In [11]:
tmp = "Āditya6 = Varuṇa I, Varuṇaṃ, Varuṇaṃ devam Āṃ patim ambhasāṃ, Varuṇo deva Āo"
triplets(tmp,256)

[{'head': 'Āditya6', 'type': 'has part', 'tail': 'Varuṇa I'},
 {'head': 'Āditya6', 'type': 'has part', 'tail': 'Varuṇaṃ'},
 {'head': 'Varuṇa I', 'type': 'part of', 'tail': 'Āditya6'}]

In [12]:
tmp = "Aditya6 = Varuna I, Varunam, Varunam devam Am patim ambhasam, Varuno deva Ao"
triplets(tmp,256)

[{'head': 'Aditya6', 'type': 'has part', 'tail': 'Varuna I'},
 {'head': 'Varuna I', 'type': 'part of', 'tail': 'Aditya6'},
 {'head': 'Aditya6', 'type': 'has part', 'tail': 'Varunam'},
 {'head': 'Aditya6',
  'type': 'has part',
  'tail': 'Varunam devam Am patim ambhasam'},
 {'head': 'Aditya6', 'type': 'has part', 'tail': 'Varuno deva Ao'},
 {'head': 'Varunam', 'type': 'part of', 'tail': 'Aditya6'},
 {'head': 'Varunam devam Am patim ambhasam',
  'type': 'part of',
  'tail': 'Aditya6'},
 {'head': 'Varuno deva Ao', 'type': 'part of', 'tail': 'Aditya6'}]

# Transliterating the entire cleaned corpus before feeding into the model

In [1]:
def transliterate_to_latin(word):
    iast_mapping = {
        'á': 'aa',
        'ñ': 'n',
        'ā': 'aa',
        'ă': 'aa',
        'ī': 'i',
        'ĭ': 'i',
        'Ś': 'sh',
        'ś': 'sh',
        'Ū': 'u',
        'ū': 'u',
        'ḍ': 'd',
        'Ḣ': '',
        'ḥ': '',
        'ṃ': 'm',
        'ṅ': 'n',
        'ṇ': 'n',
        'ṛ': 'ri',
        'ṝ': 'ri',
        'ṣ': 'sh',
        'ṭ': 't'
    }

    latin_word = ''
    for char in word:
        if char in iast_mapping:
            latin_word += iast_mapping[char]
        else:
            latin_word += char
    return latin_word



In [4]:
trans_data = []
for line in texts:
    temp = ""
    for word in line:
        temp = temp + transliterate_to_latin(word.lower())
    trans_data.append(temp)


In [5]:
len(trans_data)

12647

**Performing better with transliterated data**

In [9]:
tmp = "abhimanyu, son of arjuna and subhadraa, sister of krishna, is an incarna tion of varcas, son of soma, wherefore he enters soma after death, after his karman has elapsed   etymology of his name  he is the vamshakara of the paandavas  in the  beginning of the exile of the paandavas he is carried to dvaarakaa with his mother by krishna, where he is educated and protected by raukmineya  after the elapse of the exile he joins the paandavas in upaplavya with krishna and subhadraa, and is there married to uttaraa, daughter of king viraata  he fights bravely in the great battle, until he on the th day, when enjoined by yudhishthira to break dronas cakravyuha, is surrounded and killed by jayadratha and his people, leaving his wife uttaraa with a yet unborn son, parikshit, the only one left to propagate the race of the paandavas at first he is not seen in heaven by yudhishthira, but later he is shown to him beside soma in heaven, l, g, st day of the battle, nd day of the battle, rd day, th day, th day, th day, th day, th day, th day, th day, th day, th day, th day, cf aarjuni, saubhadra, kaarshni, arjunaatmaja, arjunaavara, phaalguni, shakraatmajaatmaja"
triplets(tmp,256)

NameError: name 'triplets' is not defined

In [10]:
tmp = "Abhimanyu, son of Arjuna and Subhadrā, sister of Kṛṣṇa, is an incarna tion of Varcas, son of Soma, wherefore he enters Soma after death, after his Karman has elapsed   etymology of his name  he is the vaṃśakara of the Pāṇḍavas  In the  beginning of the exile of the Pāṇḍavas he is carried to Dvārakā with his mother by Kṛṣṇa, where he is educated and protected by Raukmiṇeya  After the elapse of the exile he joins the Pāṇḍavas in Upaplavya with Kṛṣṇa and Subhadrā, and is there married to Uttarā, daughter of king Virāṭa  He fights bravely in the great battle, until he on the th day, when enjoined by Yudhiṣṭhira to break Droṇas cakravyūha, is surrounded and killed by Jayadratha and his people, leaving his wife Uttarā with a yet unborn son, Parikṣit, the only one left to propagate the race of the Pāṇḍavas At first he is not seen in Heaven by Yudhiṣṭhira, but later he is shown to him beside Soma in Heaven, l, g, st day of the battle, nd day of the battle, rd day, th day, th day, th day, th day, th day, th day, th day, th day, th day, th day, Cf Ārjuni, Saubhadra, Kārṣṇi, Arjunātmaja, Arjunāvara, Phālguni, Śakrātmajātmaja"
triplets(tmp,256)

NameError: name 'triplets' is not defined

In [199]:
print(tmp)

Abhimanyu, son of Arjuna and Subhadrā, sister of Kṛṣṇa, is an incarna tion of Varcas, son of Soma, wherefore he enters Soma after death, after his Karman has elapsed   etymology of his name  he is the vaṃśakara of the Pāṇḍavas  In the  beginning of the exile of the Pāṇḍavas he is carried to Dvārakā with his mother by Kṛṣṇa, where he is educated and protected by Raukmiṇeya  After the elapse of the exile he joins the Pāṇḍavas in Upaplavya with Kṛṣṇa and Subhadrā, and is there married to Uttarā, daughter of king Virāṭa  He fights bravely in the great battle, until he on the th day, when enjoined by Yudhiṣṭhira to break Droṇas cakravyūha, is surrounded and killed by Jayadratha and his people, leaving his wife Uttarā with a yet unborn son, Parikṣit, the only one left to propagate the race of the Pāṇḍavas At first he is not seen in Heaven by Yudhiṣṭhira, but later he is shown to him beside Soma in Heaven, l, g, st day of the battle, nd day of the battle, rd day, th day, th day, th day, th da

# Extracting "=" Relations

In [6]:
import re

def extract_same_and_remove(text_list):
    pattern = r'(\b\w+\b)\s*=\s*(\b\w+\b)'
    relations = []
    cleaned_texts = []
    for text in text_list:
        matches = re.findall(pattern, text)
        if matches:
            for match in matches:
                relation = {
                    'head': match[0],
                    'type': 'is same as',
                    'tail': match[1]
                }
                relations.append(relation)
        else:
            cleaned_texts.append(text.strip())
    return relations, cleaned_texts


In [7]:
relations, trans_data = extract_same_and_remove(trans_data)

In [8]:
print(len(relations))
(relations[10:15])

5386


[{'head': 'aacaarya1', 'type': 'is same as', 'tail': 'drona'},
 {'head': 'aacaarya2', 'type': 'is same as', 'tail': 'kripa'},
 {'head': 'aacaarya3', 'type': 'is same as', 'tail': 'parashuraama'},
 {'head': 'aacaaryamukhya', 'type': 'is same as', 'tail': 'drona'},
 {'head': 'aacaaryanandana', 'type': 'is same as', 'tail': 'ashvatthaaman'}]

In [9]:
relations

[{'head': 'abhagnayoga', 'type': 'is same as', 'tail': 'no'},
 {'head': 'abhaasura', 'type': 'is same as', 'tail': 'no'},
 {'head': 'abhigamya', 'type': 'is same as', 'tail': 'shiva'},
 {'head': 'bhaaryaa', 'type': 'is same as', 'tail': 'uttaraa'},
 {'head': 'abhimanyuvadha', 'type': 'is same as', 'tail': 'abhimanyuvadha'},
 {'head': 'abhiraama', 'type': 'is same as', 'tail': 'shiva'},
 {'head': 'abhivaadya', 'type': 'is same as', 'tail': 'shiva'},
 {'head': 'acala3', 'type': 'is same as', 'tail': 'the'},
 {'head': 'acala4', 'type': 'is same as', 'tail': 'vishnu'},
 {'head': 'acalopama', 'type': 'is same as', 'tail': 'shiva'},
 {'head': 'aacaarya1', 'type': 'is same as', 'tail': 'drona'},
 {'head': 'aacaarya2', 'type': 'is same as', 'tail': 'kripa'},
 {'head': 'aacaarya3', 'type': 'is same as', 'tail': 'parashuraama'},
 {'head': 'aacaaryamukhya', 'type': 'is same as', 'tail': 'drona'},
 {'head': 'aacaaryanandana', 'type': 'is same as', 'tail': 'ashvatthaaman'},
 {'head': 'aacaaryaputra

# Removing such sentences which have less than 10 words because it might confuse the model

In [10]:
print(len(trans_data))
trans_data[:10]

7347


['abala .',
 'abbhakshaa .',
 'abhagnaparisankhyaana, the great spirit. b, .',
 'aabhaasuraa, a class of gods .',
 'abhaya .',
 'abhibhu, king of kaashi vibhu. = krishna, .',
 'abhijit, the nd of the nakshatras, when beginning with ashvini .',
 'abhijita, the eighth hour of the day . a nakshatra .',
 'abhimanyu, son of arjuna and subhadraa, sister of krishna, is an incarna tion of varcas, son of soma, wherefore he enters soma after death, after his karman has elapsed etymology of his name he is the vamshakara of the paandavas . in the beginning of the exile of the paandavas he is carried to dvaarakaa with his mother by krishna, where he is educated and protected by raukmineya . after the elapse of the exile he joins the paandavas in upaplavya with krishna and subhadraa, and is there married to uttaraa, daughter of king viraata . he fights bravely in the great battle, until he on the when enjoined by yudhishthira to break dronas cakravyuha, is surrounded and killed by jayadratha and his

In [11]:
def remove_short_strings(text_list):
    cleaned_texts = []
    for text in text_list:
        if len(text.split()) >= 3:  # Check if the string has at least 10 words
            cleaned_texts.append(text.strip())
    return cleaned_texts
    

In [12]:
trans_data = remove_short_strings(trans_data)
print(len(trans_data))
trans_data[:6]


6571


['abhagnaparisankhyaana, the great spirit. b, .',
 'aabhaasuraa, a class of gods .',
 'abhibhu, king of kaashi vibhu. = krishna, .',
 'abhijit, the nd of the nakshatras, when beginning with ashvini .',
 'abhijita, the eighth hour of the day . a nakshatra .',
 'abhimanyu, son of arjuna and subhadraa, sister of krishna, is an incarna tion of varcas, son of soma, wherefore he enters soma after death, after his karman has elapsed etymology of his name he is the vamshakara of the paandavas . in the beginning of the exile of the paandavas he is carried to dvaarakaa with his mother by krishna, where he is educated and protected by raukmineya . after the elapse of the exile he joins the paandavas in upaplavya with krishna and subhadraa, and is there married to uttaraa, daughter of king viraata . he fights bravely in the great battle, until he on the when enjoined by yudhishthira to break dronas cakravyuha, is surrounded and killed by jayadratha and his people, leaving his wife uttaraa with a y

# Writing the transliterated and cleaned file for feeding into the model

In [13]:
with open("Transliterated_cleaned.txt", "w",encoding='utf-8') as file:
    # Iterate through the list and write each string to the file
    for string in trans_data:
        file.write(string + "\n")  # Add a newline character to separate strings

print("List of strings has been written to the file.")

List of strings has been written to the file.
